In [ ]:
%pip install comet_ml --quiet
import comet_ml

Install requirements

In [ ]:
# Check version of packages
import torch
print(torch)

import torchvision
print(torchvision)

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

Check Colab GPU and CPU Setup

In [ ]:
# Check GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# Check CPU info
!lscpu |grep 'Model name'

## Setup

In [ ]:
%%capture
if not os.path.exists("images.tar.gz"):
  # Download dataset
  !wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz

  # Download truth label
  !wget https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
  # Unpack the dataset and ground truth
  filename_images = "images.tar.gz"
  !tar -zxvf {filename_images}

  filename_annotations = "annotations.tar.gz"
  !tar -zxvf {filename_annotations}

## CATEGORIZE DATA AFTER ANIMAL BREED (37 IN TOTAL)

In [ ]:
import os
import shutil
from collections import defaultdict
import random
SEED_FOR_DATA_SPLIT = 13948571
temp_state_file = "temp_state_file"
do_img_move = True

if os.path.exists(temp_state_file):
  with open(temp_state_file, "r") as f:
    for line in f:
      if "FILES_ALREADY_MOVED" in line:
        do_img_move = False
        break

if do_img_move:
  # 37 classes
  annotations_file = "annotations/list.txt"
  images_base_path = "images/"
  res_base_dir = "sorted_imgs"
  classes = defaultdict(list)
  with open(annotations_file) as f:
      for line in f:
          if line.startswith("#"):
              continue
          name, class_id, species, _ = line.strip().split(" ")
          classes[int(class_id)].append(name + ".jpg")

  for class_id, files in classes.items():
      os.makedirs(f"{res_base_dir}/{str(class_id)}/", exist_ok=True)
      
      number_of_files = len(files)
      for i in range(number_of_files):
          shutil.move(os.path.join(images_base_path, files[i]), f"{res_base_dir}/{str(class_id)}/{files[i]}")

  with open(temp_state_file, "a") as f:
    f.write("FILES_ALREADY_MOVED")


In [ ]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# Initial inspiration: https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets 
def dataset_split(dataset, ratio_to_first_split):
  first_indices, second_indices, _, _ = train_test_split(
        range(len(dataset)),
        dataset.targets,
        stratify=dataset.targets,
        train_size=ratio_to_first_split,
        random_state=SEED_FOR_DATA_SPLIT
      )
  
  return first_indices, second_indices


def subset_split(dataset, subset_indices, ratio_to_first_split):
  target_labels = list(map(lambda i: dataset.targets[i], subset_indices))
  
  first_indices, second_indices, _, _ = train_test_split(
        subset_indices,
        target_labels,
        stratify=target_labels,
        train_size=ratio_to_first_split,
        random_state=SEED_FOR_DATA_SPLIT
      )
  
  return first_indices, second_indices


def _get_datasets(ratio_in_labled_train_dataset, transforms_dict):
  """Return format: test_dataset, val_dataset, labled_train_dataset, unlabled_train_dataset """
  
  labled_train_dataset = datasets.ImageFolder("sorted_imgs", transforms_dict["labled"])
  unlabled_train_dataset = datasets.ImageFolder("sorted_imgs", transforms_dict["unlabled"])
  test_and_val_dataset = datasets.ImageFolder("sorted_imgs", transforms_dict["test_and_val"])

  train_and_val_indices, test_indices = dataset_split(labled_train_dataset, ratio_to_first_split=0.9)
  train_indices, val_indices = subset_split(labled_train_dataset, train_and_val_indices, ratio_to_first_split=0.75)
  
  test_dataset = Subset(test_and_val_dataset, test_indices)
  val_dataset = Subset(test_and_val_dataset, val_indices)

  if ratio_in_labled_train_dataset == 1.0:
    return test_dataset, val_dataset, Subset(labled_train_dataset, train_indices), None

  train_labled_indices, train_unlabled_indices = subset_split(labled_train_dataset, train_indices, ratio_to_first_split=ratio_in_labled_train_dataset)
  return test_dataset, val_dataset, Subset(labled_train_dataset, train_labled_indices), Subset(unlabled_train_dataset, train_unlabled_indices)


def get_dataloaders_dict(ratio_in_labled_train_dataset, transforms_dict, batch_size):
  """ Note: currently test set is fixed at 10 %, val at 20 % and rest (labled + unlabled) is 70 %"""

  test_dataset, val_dataset, labled_train, unlabled_train = _get_datasets(ratio_in_labled_train_dataset, transforms_dict)
  dataloaders_dict = {
    "test": torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2),
    "val": torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2),
    "labled": torch.utils.data.DataLoader(labled_train, batch_size=batch_size, shuffle=True, num_workers=2),
  }
  # number of supervised and unsupervised batches will be the same. This will make looping through the dataloader easier
  nbr_supervised_batches = len(dataloaders_dict["labled"].batch_sampler)
  nbr_unsupervised_samples = len(unlabled_train.indices)
  unsupervised_batch_size = round(nbr_unsupervised_samples / nbr_supervised_batches)
  dataloaders_dict["unlabled"] = None if not unlabled_train else torch.utils.data.DataLoader(unlabled_train, batch_size=unsupervised_batch_size, shuffle=True, num_workers=2)
  
  return dataloaders_dict


input_size = 224
data_transforms = {
    'labled': transforms.Compose([
        transforms.RandomResizedCrop(input_size,scale=(0.9,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'unlabled': transforms.Compose([
        transforms.RandomResizedCrop(input_size,scale=(0.9,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test_and_val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataloaders_dict = get_dataloaders_dict(ratio_in_labled_train_dataset=0.9, transforms_dict=data_transforms, batch_size=9)
print(dataloaders_dict.keys())
# NOTE: there is a new way to get number of batches:
# Ex. len(dataloaders_dict["test"].batch_sampler)


Download model

### Finetune resnet-18

In [ ]:
from collections import Counter
target_labels = list(map(lambda i: dataloaders_dict["labled"].dataset.dataset.targets[i], dataloaders_dict["labled"].dataset.indices))
print(Counter(target_labels))
#len(dataloaders_dict["unlabled"].dataset.indices)
print(len(dataloaders_dict["labled"].dataset), len(dataloaders_dict["unlabled"].dataset))
print(len(dataloaders_dict["labled"].batch_sampler), len(dataloaders_dict["unlabled"].batch_sampler))
dataloaders_dict["labled"].batch_size, dataloaders_dict["unlabled"].batch_size

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:

print(device)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name("cuda:0"))

Helper functions

In [ ]:
def test_acc(model, dataloader_dict):
  model.eval()
  running_corrects = 0
  for i, (inputs, labels) in enumerate(dataloader_dict["test"]):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, pred_for_labled_data = torch.max(outputs, 1)                    
    running_corrects += torch.sum(pred_for_labled_data == labels.data)
  test_acc = running_corrects.double() / len(dataloader_dict["test"].dataset)
  
  return test_acc.item()
  

In [ ]:
def load_resnet_with_unfrozen_layers(num_unfrozen_layers, unfreeze_bn, num_classes=37):
  model_ft = models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

  for name, param in model_ft.named_parameters():
    if not any(map(lambda layer_str: layer_str in name, [f"layer{4-i}" for i in range(num_unfrozen_layers)])) or (not unfreeze_bn and "bn" in name):
      param.requires_grad = False

  num_ftrs = model_ft.fc.in_features
  model_ft.fc = nn.Linear(num_ftrs, num_classes)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model_ft = model_ft.to(device)

  return model_ft

In [ ]:
def load_cifar_resnet():
    num_classes = 10
    model_ft = models.resnet18()
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    model_ft = model_ft.to(device)

    return model_ft

In [ ]:

def alpha_scheduler(current_epoch_num, T1, T2, max_alpha):
  # PART 1: [ALPHA = 0]
  if (current_epoch_num) < (T1): 
    return 0

  # PART 2: epoch "T1" ---> epoch "T2"
  elif (current_epoch_num) < (T2):
    return (current_epoch_num-T1)/(T2-T1) * max_alpha
  
  # PART 3: [ALPHA = max_alpha]
  return max_alpha


def train_model_pseudo_labeling(model, dataloaders, criterion, optimizers, schedulers, experiment):
    since = time.time()
    num_epochs = experiment.get_parameter("num_epochs")
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    step = -1
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        # training phase
        phase = "train"
        model.train()  # Set model to training mode
        #alpha = alpha_scheduler(epoch, T1=experiment.get_parameter("T1"), T2=experiment.get_parameter("T2"), max_alpha=experiment.get_parameter("max_alpha")) # Update alpha
        #print('ALPHA =', alpha)
        alpha = experiment.get_parameter("fixed-alpha")

        running_loss = 0.0
        running_corrects = 0

        # Create tqdm progress bar object
        print(f"Processing {phase} data")

        # Iterate over data.
        
        for (labled_input, labels), (unlabled_input, secret_labels) in zip(dataloaders["labled"], dataloaders["unlabled"]):
            step += 1
            labled_input = labled_input.to(device)
            labels = labels.to(device)
            unlabled_input = unlabled_input.to(device)
            secret_labels = secret_labels.to(device)

            # zero the parameter gradients
            for optimizer in optimizers:
              optimizer.zero_grad()

            labling_confidence = experiment.get_parameter("labling_confidence")
            with torch.set_grad_enabled(True):
                # supervised forward pass
                supervised_forward_pass_outputs = model(labled_input)
                _, pred_for_labled_data = torch.max(supervised_forward_pass_outputs, 1)
                supervised_loss = criterion(supervised_forward_pass_outputs, labels)

                # pseudo labeled forward pass
                unsupervised_out = model(unlabled_input)
                softmax_unsupervised_out = nn.functional.softmax(unsupervised_out, dim=1)
                max_probs, pseudo_labels = torch.max(softmax_unsupervised_out, dim=1)
                # only use pseudolables with high enough confidence
                pseudo_label_loss = criterion(unsupervised_out[max_probs >= labling_confidence], pseudo_labels[max_probs >= labling_confidence])
                confident_labels = (max_probs >= labling_confidence).sum().item()
                with torch.no_grad():
                  in_correct_label_count = (pseudo_labels[max_probs >= labling_confidence] != secret_labels[max_probs >= labling_confidence]).sum().item()
                  experiment.log_metric("confident pseudo labels", confident_labels, step=step, epoch=epoch)
                  experiment.log_metric("incorrect pseudo labels ratio", in_correct_label_count / max(1, confident_labels), step=step, epoch=epoch)
                if pseudo_label_loss.isnan().any():
                  pseudo_label_loss = 0

                # combined loss
                loss = supervised_loss + alpha * pseudo_label_loss

                # backward + optimize only if in training phase
                loss.backward()
                for optimizer in optimizers:
                  optimizer.step()
                for scheduler in schedulers:
                  if type(scheduler) == lr_scheduler.OneCycleLR:
                    scheduler.step()

            experiment.log_metric("unsupervised loss", pseudo_label_loss, step=step, epoch=epoch)
            experiment.log_metric("scaled unsupervised loss", alpha * pseudo_label_loss, step=step, epoch=epoch)
            experiment.log_metric("supervised loss", supervised_loss, step=step, epoch=epoch)
            experiment.log_metric("alpha", alpha, step=step, epoch=epoch)
            # statistics
            running_loss += loss.item()
            running_corrects += torch.sum(pred_for_labled_data == labels.data)


        epoch_loss = running_loss / len(dataloaders["labled"].batch_sampler) # note len(dataloaders[phase].dataset) is nbr of samples not nbr of batches
        epoch_acc = running_corrects.double() / len(dataloaders["labled"].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        experiment.log_metric(f"train accuracy", epoch_acc, step=step, epoch=epoch)

        for scheduler in schedulers:
          if scheduler is None or scheduler == lr_scheduler.OneCycleLR:
            continue
          if type(scheduler) == lr_scheduler.ReduceLROnPlateau:
            scheduler.step(epoch_acc)
          else:
            scheduler.step()

        # validaiton phase starts here

        phase = "val"
        model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Create tqdm progress bar object
        print(f"Processing {phase} data")

        # Iterate over data.
        for (labled_input, labels) in dataloaders["val"]:
            labled_input = labled_input.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            for optimizer in optimizers:
              optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                # Get model outputs and calculate loss
                  supervised_forward_pass_outputs = model(labled_input)
                  _, pred_for_labled_data = torch.max(supervised_forward_pass_outputs, 1)
                  loss = criterion(supervised_forward_pass_outputs, labels)
                    
            # statistics
            running_loss += loss.item()
            running_corrects += torch.sum(pred_for_labled_data == labels.data)

        epoch_loss = running_loss / len(dataloaders["val"].batch_sampler) # note len(dataloaders[phase].dataset) is nbr of samples not nbr of batches
        epoch_acc = running_corrects.double() / len(dataloaders["val"].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # deep copy the model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        val_acc_history.append(epoch_acc)
        experiment.log_metric(f"val accuracy", epoch_acc, step=step, epoch=epoch)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    experiment.log_metric("best model test acc", test_acc(model, dataloaders_dict))
    experiment.end()

    return model, val_acc_history

Load data

Cell for experimentation

Run Training and Validation Step


In [ ]:
hyperparams = {
  "batch_size": 9, 
  "num_epochs": 15,
  #"T1": 6,
  #"T2": 12,
  #"max_alpha": 3,
  "fixed-alpha": 1,
  "labling_confidence": 0.95,
  "lr": 0.001,
  "weight_decay": 0.005,
  "ratio_of_labled_data": 0.25,
  "seed": SEED_FOR_DATA_SPLIT,
  "unfrozen_layers": 0,
}

runs = [
    ("ratio_of_labled_data", 0.9),
    ("ratio_of_labled_data", 0.75),
    ("ratio_of_labled_data", 0.5),
    ("ratio_of_labled_data", 0.25),
    ("ratio_of_labled_data", 0.10),
    ("ratio_of_labled_data", 0.02),
]

for run in runs:
  hyperparams_copy = hyperparams.copy()
  hyperparams_copy[run[0]] = run[1]
  experiment = comet_ml.Experiment(
    api_key="API-KEY-HERE", 
    project_name="dd2424-project",
    workspace="storsorken",
  )
  #for experiment in opt.get_experiments():
  experiment.add_tag("TAG-HERE")

  experiment.log_parameters(hyperparams_copy)

  unfrozen_layers = 0
  model_ft = model_ft = load_resnet_with_unfrozen_layers(experiment.get_parameter("unfrozen_layers"), True, num_classes=37)


  # creates a dict that groups model params into a dict with lists.
  # all params for ex. layer4 can be accessed using layer_param_dict["layer4"]

  layer_param_dict = defaultdict(list)
  for name,param in model_ft.named_parameters():
    if param.requires_grad:
      layer_param_dict[name.split(".")[0]].append(param)

  all_params = []
  for key, param_list in layer_param_dict.items():
    all_params += param_list

  print(layer_param_dict.keys())

  optimizers = [
      optim.Adam(model_ft.parameters(), lr=experiment.get_parameter("lr"), weight_decay=experiment.get_parameter("weight_decay")),
      #optim.Adam(layer_param_dict["layer4"], lr=0.001, weight_decay=1e-5),
      #optim.Adam(layer_param_dict["layer3"], lr=0.001),
  ]
  schedulers = [
      lr_scheduler.ReduceLROnPlateau(optimizers[0], patience=2),
      #lr_scheduler.LambdaLR(optimizers[1], lr_lambda=lambda epoch: 0.95 ** epoch),
      #lr_scheduler.LambdaLR(optimizers[2], lr_lambda=lambda epoch: 0.95 ** epoch),
  ]

  dataloaders_dict = get_dataloaders_dict(experiment.get_parameter("ratio_of_labled_data"), data_transforms, experiment.get_parameter("batch_size"))


  # Setup the loss fxn
  criterion = nn.CrossEntropyLoss()

  # Train and evaluate
  model_ft, hist = train_model_pseudo_labeling(model_ft, dataloaders_dict, criterion, optimizers, schedulers, experiment)
  #model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizers, schedulers, num_epochs=num_epochs)
  test_acc_res = test_acc(model_ft, dataloaders_dict)
  print(f"Test acc: {test_acc_res}")
  with open("confident_pseudo_labels.txt", "a") as f:
    line = f"ratio_of_labled_data: {str(run[1])}. Test acc: {str(test_acc_res)}. Val: {list(map(lambda x: x.item(), hist))}\n"
    f.write(line)

# plot results
plt.plot(range(1, len(hist) + 1), list(map(lambda x: x.item(), hist)))
plt.xlabel("Epoch")
plt.ylabel("Val accuracy")
plt.show()

In [ ]:
# NOTE: run this line very sparinlgly
#print(f"Test acc: {test_acc(model_ft, dataloaders_dict)}")

In [ ]:
#len(dataloaders_dict["val"].batch_sampler), (dataloaders_dict["val"].batch_size), (dataloaders_dict["val"])